# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
import datetime
from scipy import stats #The SciPy stats module

In [2]:
import api_token  # handle iexcloud API tokens

env = "sandbox"  # "cloud"
BASE_URL, IEX_CLOUD_API_TOKEN = api_token.get_base_url(env=env), api_token.get_api_token(env=env)

In [3]:
TOP_N = 100
portfolio_size = 1000000

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [4]:
stock_file = 'sp_500_stocks.csv.gz'
stocks = pd.read_csv(stock_file, compression="gzip")

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [5]:
symbol = 'AAPL'
api_url =  f'{BASE_URL}/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'LAAS)MNC GND TK BTQEGLLEA/AR(ESOS',
 'calculationPrice': 'close',
 'open': 124.61,
 'openTime': 1631218037173,
 'openSource': 'fofcaili',
 'close': 124.52,
 'closeTime': 1629176000178,
 'closeSource': 'clioafif',
 'high': 126.84,
 'highTime': 1608312393852,
 'highSource': 'ui et ned15y ldaceepmri',
 'low': 125.79,
 'lowTime': 1670018440730,
 'lowSource': 'eeimade  dti1n5eupcly r',
 'latestPrice': 122.74,
 'latestSource': 'Close',
 'latestTime': 'December 11, 2020',
 'latestUpdate': 1674917466888,
 'latestVolume': 88075546,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexLastUpdated': None,
 'delayedPrice': 122.68,
 'delayedPriceTime': 1684550650934,
 'oddLotDelayedPrice': 128.259,
 'oddLotDelayedPriceTime': 1658143773033,
 'extendedPrice': 123.99,
 'extendedChange': -0.14,
 'extendedChangePercent': -0.00115,
 'extendedPriceTime': 1614504855153,
 'previousClose': 125.2,
 'previousVolume': 83962550,
 'change':

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [6]:
pe_ratio = data['peRatio']
pe_ratio

37.38

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [7]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [8]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url =  f'{BASE_URL}/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,122.88,55.55,N/A
1,AAL,17.20,-1.24,N/A
2,AAP,164.01,23.76,N/A
3,AAPL,125.90,38.69,N/A
4,ABBV,110.07,24,N/A
...,...,...,...,...
500,YUM,110.59,31.88,N/A
501,ZBH,142.15,-192.73,N/A
502,ZBRA,395.04,43.29,N/A
503,ZION,42.65,17.85,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [9]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]   # filter out stock w/o positive earnings
final_dataframe = final_dataframe[:(TOP_N+1)]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)

## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [10]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [11]:
# portfolio_input()

You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [12]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,33.93,2.08,291
1,NLOK,19.92,3.76,497
2,BIO,591.90,5.05,16
3,UNM,23.61,5.13,419
4,KIM,15.14,6.7,653
...,...,...,...,...
96,DVA,108.32,17.7,91
97,CAG,35.68,17.75,277
98,HSIC,72.92,17.78,135
99,SIVB,361.00,17.82,27


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio (PE)
* Price-to-book ratio (PB)
* Price-to-sales ratio (PS)
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [13]:
symbol = 'AAPL'
batch_api_call_url =  f'{BASE_URL}/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [14]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url =  f'{BASE_URL}/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )

## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [15]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,211.32,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,230.69,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.00,N/A,10.16,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,25.85,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,49.60,N/A,14.61,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
168,EVRG,57.88,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,29.57,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRC,130.11,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
194,FTI,9.77,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,1822.48,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, __we will replace missing data with the average non-`NaN` data point from that column.__

Here is the code to do this:

In [16]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [17]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [18]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# # Print each percentile score to make sure it was calculated properly
# for metric in metrics.values():
#     print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,120.56,N/A,54.24,0.863366,7.5700,0.728713,7.0700,0.811881,33.150912,0.853465,13.540252,0.821782,N/A
1,AAL,17.74,N/A,-1.26,0.141584,-1.9500,0.0455446,0.4252,0.0316832,-6.116196,0.0277228,1.445003,0.0613861,N/A
2,AAP,162.48,N/A,23.23,0.437624,2.9600,0.414851,1.1300,0.178218,10.859817,0.261386,2.456763,0.124752,N/A
3,AAPL,127.66,N/A,37.56,0.712871,32.9700,0.962376,7.8600,0.849505,26.819103,0.79604,20.166885,0.944554,N/A
4,ABBV,110.89,N/A,23.59,0.455446,12.4300,0.855446,4.8200,0.70297,16.477377,0.518812,9.449848,0.647525,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,107.40,N/A,32.94,0.659406,-3.9700,0.0415842,6.0100,0.766337,23.813216,0.740594,10.479834,0.69901,N/A
501,ZBH,146.65,N/A,-197.99,0.019802,2.5200,0.360396,4.2900,0.617822,25.451816,0.776238,7.343894,0.471287,N/A
502,ZBRA,393.45,N/A,43.62,0.778218,10.4300,0.831683,4.7500,0.680198,29.515831,0.833663,11.447784,0.740594,N/A
503,ZION,42.03,N/A,17.83,0.346535,0.9664,0.0930693,2.2700,0.389109,10.434049,0.239604,2.700555,0.142574,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for __Robust Value__), which is the value score that we'll use to filter for 
stocks in this investing strategy.

__The higher RV Score, the more expensive the stock is__

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [19]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,120.56,N/A,54.24,0.863366,7.5700,0.728713,7.0700,0.811881,33.150912,0.853465,13.540252,0.821782,0.815842
1,AAL,17.74,N/A,-1.26,0.141584,-1.9500,0.0455446,0.4252,0.0316832,-6.116196,0.0277228,1.445003,0.0613861,0.0615842
2,AAP,162.48,N/A,23.23,0.437624,2.9600,0.414851,1.1300,0.178218,10.859817,0.261386,2.456763,0.124752,0.283366
3,AAPL,127.66,N/A,37.56,0.712871,32.9700,0.962376,7.8600,0.849505,26.819103,0.79604,20.166885,0.944554,0.853069
4,ABBV,110.89,N/A,23.59,0.455446,12.4300,0.855446,4.8200,0.70297,16.477377,0.518812,9.449848,0.647525,0.63604
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,107.40,N/A,32.94,0.659406,-3.9700,0.0415842,6.0100,0.766337,23.813216,0.740594,10.479834,0.69901,0.581386
501,ZBH,146.65,N/A,-197.99,0.019802,2.5200,0.360396,4.2900,0.617822,25.451816,0.776238,7.343894,0.471287,0.449109
502,ZBRA,393.45,N/A,43.62,0.778218,10.4300,0.831683,4.7500,0.680198,29.515831,0.833663,11.447784,0.740594,0.772871
503,ZION,42.03,N/A,17.83,0.346535,0.9664,0.0930693,2.2700,0.389109,10.434049,0.239604,2.700555,0.142574,0.242178


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [20]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True, ascending = True)   # the lower RV score, the more potential value
rv_dataframe = rv_dataframe[:(TOP_N+1)]
rv_dataframe.reset_index(drop = True, inplace = True)

## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [21]:
# portfolio_input()

In [22]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

/home/wengong/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,23.61,419,5.04,0.154455,0.4403,0.0475248,0.4104,0.029703,2.639430,0.0336634,0.398734,0.0039604,0.0538614
1,AIG,39.64,249,-7.04,0.10495,0.5343,0.0554455,0.7462,0.0673267,4.365090,0.0415842,0.730360,0.0158416,0.0570297
2,AAL,17.74,558,-1.26,0.141584,-1.9500,0.0455446,0.4252,0.0316832,-6.116196,0.0277228,1.445003,0.0613861,0.0615842
3,MET,48.42,204,7.85,0.162376,0.5881,0.0574257,0.6716,0.0574257,4.333586,0.039604,0.638041,0.0118812,0.0657426
4,HFC,28.76,344,-28.39,0.0673267,0.8659,0.0831683,0.3130,0.0138614,6.326810,0.0633663,2.222888,0.116832,0.0689109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,BEN,25.51,388,16.00,0.29901,1.2600,0.146535,2.2100,0.375248,11.506655,0.29703,2.072411,0.106931,0.24495
97,CXO,63.94,154,-1.26,0.141584,1.6700,0.241584,3.6300,0.568317,-10.483844,0.0237624,4.495361,0.263366,0.247723
98,LKQ,36.82,268,18.70,0.366337,2.1700,0.312871,0.9726,0.124752,10.900936,0.267327,3.027506,0.172277,0.248713
99,ABC,100.88,98,12.95,0.227723,5.2000,0.621782,0.1077,0.0019802,9.031835,0.158416,4.092236,0.239604,0.249901


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [23]:
today = datetime.datetime.now()
xlsx_file = f"003-value_strategy_{today.strftime('%Y-%m-%d')}.xlsx"

In [24]:
writer = pd.ExcelWriter(xlsx_file, engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [25]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [26]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 10, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [27]:
writer.save()